In [35]:
# Import packages 
import pandas as pd
import numpy as np
import pandas.io.sql as sql
import pymysql
import thefuzz
import xlwt
import sqlalchemy
import time
import civitatisAPI as civi
import requests
import json
import difflib


from requests.auth import HTTPBasicAuth
from thefuzz import fuzz
from thefuzz import process
from sqlalchemy import create_engine
from pymysql import*
from IPython.display import clear_output


#### 1) Extract and prepare cPanel tables 

In [36]:
# cPanel credentials and connection
user= 'beder'
password="hTz8@A6Yn652"
host="server.digizonelabs.com"
databases_schema = []

con=connect(user="beder",password="hTz8@A6Yn652",host="server.digizonelabs.com",database="beder_appprod")




In [37]:
# Call tables: place & posts
posts=sql.read_sql('select * from beder_appprod.post',con)
place=sql.read_sql('select * from beder_appprod.place',con)

c:\Users\beder\.conda\envs\ACTIMPLE2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\beder\.conda\envs\ACTIMPLE2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [38]:
#Call Geo-dimensional tables
countries=sql.read_sql('select * from beder_appprod.countries',con)
regions=sql.read_sql('select * from beder_appprod.regions',con)
provinces=sql.read_sql('select * from beder_appprod.provinces',con)
cities=sql.read_sql('select * from beder_appprod.cities',con)


c:\Users\beder\.conda\envs\ACTIMPLE2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\beder\.conda\envs\ACTIMPLE2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\beder\.conda\envs\ACTIMPLE2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\beder\.conda\envs\ACTIMPLE2022\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy 

In [39]:
cities.shape

(2384, 34)

In [40]:
# Table with all destinations: Regions, provinces and cities
frames = [regions,provinces,cities]
merged_locations= pd.concat(frames)
merged_locations.columns

Index(['region_id', 'country_id', 'top_region_id', 'google_api_level',
       'google_id', 'english_name', 'spanish_name', 'top_spot_level_1',
       'top_area_level_1', 'reference_airport', 'spot_image',
       'discover_image_1', 'discover_image_2', 'discover_image_3',
       'rules_level_3', 'rules_level_4', 'sports_and_adventure', 'nature',
       'attractions', 'beach', 'food', 'nightlife', 'arts_and_culture',
       'accomodation', 'created_at', 'updated_at', 'airport_id', 'latitude',
       'longitude', 'province_id', 'sub_region_id', 'top_spot_level_2',
       'top_area_level_2', 'city_id', 'sub_province_id', 'top_spot_level_3',
       'top_area_level_3', 'spots_are_loaded'],
      dtype='object')

In [29]:
# Filter columns in all destinations table
merged_locations=merged_locations[['country_id','region_id','province_id','city_id','google_id','google_api_level','english_name','spanish_name']]
merged_locations.sample(10)

,country_id,region_id,province_id,city_id,google_id,google_api_level,english_name,spanish_name
1755,59,560.0,1301.0,1200027.0,ChIJ_cFW60iHOI4RvN_x-RCUs5U,locality,Pereira,Pereira
2296,208,1461.0,2464.0,NaN,ChIJi4YXcf1fARURtDdOgY05onM,administrative_area_level_2,Shobak Qasabah District,Distrito Shobak Qasabah
265,18,191.0,276.0,NaN,ChIJw7lqE2uDyUARzjy2oo2yCtM,administrative_area_level_1,Ialoveni,Ialoveni
1963,194,1652.0,2125.0,NaN,ChIJ3_LdAHQE4TkRTanQZ4keJTs,administrative_area_level_1,Paro,Paro
1685,197,1754.0,NaN,NaN,ChIJwSxKnH4xYTcRmdoThGIswR8,administrative_area_level_1,Tibet,Tíbet
135,15,175.0,NaN,142.0,ChIJBeWEjWlODhMRNwXdmBiZDGA,locality,Saint Paul's Bay,San Pawl il-Baħar
305,203,1634.0,2403.0,317.0,ChIJW9fYFzyahzIRrw8zJvIpc-A,natural_feature,Bunaken Island,Isla Bunaken
215,25,227.0,NaN,NaN,ChIJCxVihNKQPxMRcSMmPSQdpJE,administrative_area_level_1,Calabria,Calabria
833,108,850.0,NaN,NaN,ChIJv5wGTgUJC2YRPEInRa0WmQM,administrative_area_level_1,Pohnpei,Pohnpei
2795,201,1674.0,60142.0,NaN,ChIJKehw7VFuADMRcyJX57UcJCg,administrative_area_level_2,Camiguín,Camiguín


#### 2) CIVITATIS destinations

In [41]:
# Load destinations from CIVITATIS
df_civitatis = pd.read_csv ("C:/Users/beder/Desktop/Data 2022/Projects/CIVITATIS-IMPLEMENTATION2022/Project Files/CIVITATIS-GLOBAL-TASKS/name_matching/input/all_destinations04_08_2022.csv")
df_civitatis.rename(columns = {'id':'destinationId','name':'destination_name','photos':'destination_photo'}, inplace = True)
df_civitatis.head(10)
df_civitatis.shape

(3234, 6)

#### 3) Merge only using beder table cities

In [42]:
# Filter civitatis and cities dataframe 
civitatis_filtered = df_civitatis[['destinationId','destination_name']]
cities_filtered = cities[['city_id', 'english_name', 'spanish_name', 'country_id']]

In [43]:
cities_filtered

,city_id,english_name,spanish_name,country_id
0,1,Seville,Sevilla,1
1,2,Granada,Granada,1
2,3,Córdoba,Córdoba,1
3,4,Málaga,Málaga,1
4,5,Marbella,Marbella,1
...,...,...,...,...
2379,1200757,Limerick,Limerick,37
2380,1200758,Castlecove,Castlecove,37
2381,1200759,Guatapé,Guatapé,59
2382,1200761,Guatiza,Guatiza,1


In [44]:
# First merge: without preparation
m_1 = pd.merge(civitatis_filtered,cities_filtered, how = 'left',left_on='destination_name',right_on='english_name')


In [47]:
m_1.shape

(3252, 6)

In [49]:
m_1[m_1['english_name'].isna()]

,destinationId,destination_name,city_id,english_name,spanish_name,country_id
3,4,Menorca,NaN,NaN,NaN,NaN
7,8,New York,NaN,NaN,NaN,NaN
13,15,Hong Kong,NaN,NaN,NaN,NaN
20,22,Singapore,NaN,NaN,NaN,NaN
22,24,Tokyo,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3247,4842,Santa Fe de Mondújar,NaN,NaN,NaN,NaN
3248,4843,Pedrajas de San Esteban,NaN,NaN,NaN,NaN
3249,4844,Puerto de Carti,NaN,NaN,NaN,NaN
3250,4845,Ceiba,NaN,NaN,NaN,NaN


#### 4) Using Fuzzy matching 

In [50]:
fuzz_cities = cities_filtered
fuzz_civitatis = civitatis_filtered


In [63]:
fuzz_cities

,city_id,english_name,spanish_name,country_id
0,1,Seville,Sevilla,1
1,2,Granada,Granada,1
2,3,Córdoba,Córdoba,1
3,4,Málaga,Málaga,1
4,5,Marbella,Marbella,1
...,...,...,...,...
2379,1200757,Limerick,Limerick,37
2380,1200758,Castlecove,Castlecove,37
2381,1200759,Guatapé,Guatapé,59
2382,1200761,Guatiza,Guatiza,1


In [51]:
fuzz_civitatis

,destinationId,destination_name
0,1,Venice
1,2,Madrid
2,3,Prague
3,4,Menorca
4,5,London
...,...,...
3229,4842,Santa Fe de Mondújar
3230,4843,Pedrajas de San Esteban
3231,4844,Puerto de Carti
3232,4845,Ceiba


In [52]:
fuzz_civitatis['destination_name_match'] = fuzz_civitatis['destination_name'] 

C:\Users\beder\AppData\Local\Temp\ipykernel_9728\2385714225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzz_civitatis['destination_name_match'] = fuzz_civitatis['destination_name']


In [53]:
fuzz_civitatis['destination_name'] = fuzz_civitatis['destination_name'].apply(lambda x: difflib.get_close_matches(x, fuzz_cities['english_name']))

C:\Users\beder\AppData\Local\Temp\ipykernel_9728\2735564597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzz_civitatis['destination_name'] = fuzz_civitatis['destination_name'].apply(lambda x: difflib.get_close_matches(x, fuzz_cities['english_name']))


In [54]:
fuzz_civitatis

,destinationId,destination_name,destination_name_match
0,1,"[Venice, Vic, Venhuizen]",Venice
1,2,"[Madrid, Mérida, Mérida]",Madrid
2,3,"[Prague, Page, Portmagee]",Prague
3,4,"[Mendoza, Lorca, Glenorchy]",Menorca
4,5,"[London, Condado, Lyon]",London
...,...,...,...
3229,4842,"[Sant Llorenç de Montgai, Santa Monica, Sant J...",Santa Fe de Mondújar
3230,4843,[Vegas de San Esteban],Pedrajas de San Esteban
3231,4844,"[Puerto de la Cruz, Puerto Varas, Puerto Valla...",Puerto de Carti
3232,4845,"[Cebu, Sesimbra, Curitiba]",Ceiba


In [55]:
# rename columns 
fuzz_civitatis.rename(columns = {'destination_name':'beders_name','destination_name_match':'civitatis_name'}, inplace = True)
# create the synthetic (key) counter
fuzz_civitatis['counter'] = range(len(fuzz_civitatis))
fuzz_civitatis

C:\Users\beder\AppData\Local\Temp\ipykernel_9728\4241289277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzz_civitatis.rename(columns = {'destination_name':'beders_name','destination_name_match':'civitatis_name'}, inplace = True)
C:\Users\beder\AppData\Local\Temp\ipykernel_9728\4241289277.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuzz_civitatis['counter'] = range(len(fuzz_civitatis))


,destinationId,beders_name,civitatis_name,counter
0,1,"[Venice, Vic, Venhuizen]",Venice,0
1,2,"[Madrid, Mérida, Mérida]",Madrid,1
2,3,"[Prague, Page, Portmagee]",Prague,2
3,4,"[Mendoza, Lorca, Glenorchy]",Menorca,3
4,5,"[London, Condado, Lyon]",London,4
...,...,...,...,...
3229,4842,"[Sant Llorenç de Montgai, Santa Monica, Sant J...",Santa Fe de Mondújar,3229
3230,4843,[Vegas de San Esteban],Pedrajas de San Esteban,3230
3231,4844,"[Puerto de la Cruz, Puerto Varas, Puerto Valla...",Puerto de Carti,3231
3232,4845,"[Cebu, Sesimbra, Curitiba]",Ceiba,3232


##### Dealing with list in column beders_name

In [56]:
# Finding the type of the column
for i, l in enumerate(fuzz_civitatis['beders_name']):
    print("list",i,"is", type(l))

list 0 is <class 'list'>
list 1 is <class 'list'>
list 2 is <class 'list'>
list 3 is <class 'list'>
list 4 is <class 'list'>
list 5 is <class 'list'>
list 6 is <class 'list'>
list 7 is <class 'list'>
list 8 is <class 'list'>
list 9 is <class 'list'>
list 10 is <class 'list'>
list 11 is <class 'list'>
list 12 is <class 'list'>
list 13 is <class 'list'>
list 14 is <class 'list'>
list 15 is <class 'list'>
list 16 is <class 'list'>
list 17 is <class 'list'>
list 18 is <class 'list'>
list 19 is <class 'list'>
list 20 is <class 'list'>
list 21 is <class 'list'>
list 22 is <class 'list'>
list 23 is <class 'list'>
list 24 is <class 'list'>
list 25 is <class 'list'>
list 26 is <class 'list'>
list 27 is <class 'list'>
list 28 is <class 'list'>
list 29 is <class 'list'>
list 30 is <class 'list'>
list 31 is <class 'list'>
list 32 is <class 'list'>
list 33 is <class 'list'>
list 34 is <class 'list'>
list 35 is <class 'list'>
list 36 is <class 'list'>
list 37 is <class 'list'>
list 38 is <class 'lis

In [58]:
# Individual Columns fro all unique values
fuzz_civitatis_expanded = fuzz_civitatis["beders_name"].apply(pd.Series)

C:\Users\beder\AppData\Local\Temp\ipykernel_9728\3173263432.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fuzz_civitatis_expanded = fuzz_civitatis["beders_name"].apply(pd.Series)


In [59]:
# Synthetic (key) counter
fuzz_civitatis_expanded['counter'] = range(len(fuzz_civitatis))
# rename columns 
fuzz_civitatis_expanded.rename(columns = {0:'first_best',1:'second_best',2:'third_best'}, inplace = True)
fuzz_civitatis_expanded

,first_best,second_best,third_best,counter
0,Venice,Vic,Venhuizen,0
1,Madrid,Mérida,Mérida,1
2,Prague,Page,Portmagee,2
3,Mendoza,Lorca,Glenorchy,3
4,London,Condado,Lyon,4
...,...,...,...,...
3229,Sant Llorenç de Montgai,Santa Monica,Sant Jaume de Frontanyà,3229
3230,Vegas de San Esteban,NaN,NaN,3230
3231,Puerto de la Cruz,Puerto Varas,Puerto Vallarta,3231
3232,Cebu,Sesimbra,Curitiba,3232


In [61]:
# Merge both tables
matching_merged = pd.merge(fuzz_civitatis, fuzz_civitatis_expanded, on='counter')
matching_merged = matching_merged.drop('beders_name', axis=1)


In [62]:
matching_merged.head(50)

,destinationId,civitatis_name,counter,first_best,second_best,third_best
0,1,Venice,0,Venice,Vic,Venhuizen
1,2,Madrid,1,Madrid,Mérida,Mérida
2,3,Prague,2,Prague,Page,Portmagee
3,4,Menorca,3,Mendoza,Lorca,Glenorchy
4,5,London,4,London,Condado,Lyon
5,6,Marrakech,5,Marrakech,Marken,NaN
6,7,Dublin,6,Dublin,Dubai,Duingt
7,8,New York,7,New York City,York,NaN
8,9,Las Vegas,8,Las Vegas,La Vega,Las Galeras
9,12,Valladolid,9,Valladolid,Valladolid,Nalladai


In [150]:
# Export to csv
matching_merged.to_csv('C:/Users/beder/Desktop/Data 2022/Projects/CIVITATIS-IMPLEMENTATION2022/Project Files/CIVITATIS-GLOBAL-TASKS/name_matching/output/matching_names.csv')

##### Recover city_id from manual cleaning


In [17]:
# Read table from excel
clean_manual=pd.read_excel ('C:/Users/beder/Desktop/Data 2022/Projects/CIVITATIS-IMPLEMENTATION2022/Project Files/CIVITATIS-GLOBAL-TASKS/name_matching/input/matching_civitatis_beder_v2.xlsx')
clean_manual

,Unnamed: 0,destinationId,civitatis_name,final_name,first_best,second_best,third_best,true/false,counter,ambiguous
0,0,1,Venice,Venice,Venice,Vic,Venhuizen,True,0,NaN
1,1,2,Madrid,Madrid,Madrid,Mérida,Mérida,True,1,NaN
2,2,3,Prague,Prague,Prague,Page,Pirque,True,2,NaN
3,3,4,Menorca,Ciutadella,Mendoza,Lorca,Glenorchy,False,3,NaN
4,4,5,London,London,London,Condado,Lyon,True,4,NaN
...,...,...,...,...,...,...,...,...,...,...
3229,3229,4842,Santa Fe de Mondújar,NaN,Sant Llorenç de Montgai,Santa Monica,Sant Jaume de Frontanyà,False,3229,NaN
3230,3230,4843,Pedrajas de San Esteban,NaN,Vegas de San Esteban,NaN,NaN,False,3230,NaN
3231,3231,4844,Puerto de Carti,NaN,Puerto de la Cruz,Puerto Varas,Puerto Vallarta,False,3231,NaN
3232,3232,4845,Ceiba,NaN,Cebu,Sesimbra,Curitiba,False,3232,NaN


In [64]:
clean_manual.count()

Unnamed: 0        3234
destinationId     3234
civitatis_name    3234
final_name         783
first_best        2741
second_best       2194
third_best        1747
true/false        3234
counter           3234
ambiguous            6
dtype: int64

In [19]:
clean_manual['true/false'].value_counts()

False    2575
True      659
Name: true/false, dtype: int64

In [65]:
# Merge fuzz_cities (to recover city_id) and clean_manual
final_table= pd.merge( clean_manual, fuzz_cities, how = 'left',left_on='final_name',right_on='english_name')
final_table

,Unnamed: 0,destinationId,civitatis_name,final_name,first_best,second_best,third_best,true/false,counter,ambiguous,city_id,english_name,spanish_name,country_id
0,0,1,Venice,Venice,Venice,Vic,Venhuizen,True,0,NaN,213.0,Venice,Venecia,25.0
1,1,2,Madrid,Madrid,Madrid,Mérida,Mérida,True,1,NaN,29.0,Madrid,Madrid,1.0
2,2,3,Prague,Prague,Prague,Page,Pirque,True,2,NaN,100054.0,Prague,Praga,13.0
3,3,4,Menorca,Ciutadella,Mendoza,Lorca,Glenorchy,False,3,NaN,300078.0,Ciutadella,Ciutadella,1.0
4,4,5,London,London,London,Condado,Lyon,True,4,NaN,100017.0,London,Londres,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3248,3229,4842,Santa Fe de Mondújar,NaN,Sant Llorenç de Montgai,Santa Monica,Sant Jaume de Frontanyà,False,3229,NaN,NaN,NaN,NaN,NaN
3249,3230,4843,Pedrajas de San Esteban,NaN,Vegas de San Esteban,NaN,NaN,False,3230,NaN,NaN,NaN,NaN,NaN
3250,3231,4844,Puerto de Carti,NaN,Puerto de la Cruz,Puerto Varas,Puerto Vallarta,False,3231,NaN,NaN,NaN,NaN,NaN
3251,3232,4845,Ceiba,NaN,Cebu,Sesimbra,Curitiba,False,3232,NaN,NaN,NaN,NaN,NaN


In [66]:
final_table=final_table[['destinationId','civitatis_name','final_name','english_name','city_id']]
final_table

,destinationId,civitatis_name,final_name,english_name,city_id
0,1,Venice,Venice,Venice,213.0
1,2,Madrid,Madrid,Madrid,29.0
2,3,Prague,Prague,Prague,100054.0
3,4,Menorca,Ciutadella,Ciutadella,300078.0
4,5,London,London,London,100017.0
...,...,...,...,...,...
3248,4842,Santa Fe de Mondújar,NaN,NaN,NaN
3249,4843,Pedrajas de San Esteban,NaN,NaN,NaN
3250,4844,Puerto de Carti,NaN,NaN,NaN
3251,4845,Ceiba,NaN,NaN,NaN


In [68]:
filt_final_table =final_table[final_table['final_name'].notnull()] 
filt_final_table

,destinationId,civitatis_name,final_name,english_name,city_id
0,1,Venice,Venice,Venice,213.0
1,2,Madrid,Madrid,Madrid,29.0
2,3,Prague,Prague,Prague,100054.0
3,4,Menorca,Ciutadella,Ciutadella,300078.0
4,5,London,London,London,100017.0
...,...,...,...,...,...
3217,4811,Neuquén,Neuquén,Neuquén,885.0
3219,4813,Kaneohe,Kaneohe,Kaneohe,736.0
3236,4830,Loreto,Loreto,Loreto,1200080.0
3244,4838,Cuiabá,Cuiabá,Cuiabá,971.0


In [70]:
# Export final table 
filt_final_table.to_csv('C:/Users/beder/Desktop/Data 2022/Projects/CIVITATIS-IMPLEMENTATION2022/Project Files/CIVITATIS-GLOBAL-TASKS/name_matching/output/final_matching.csv')